In [39]:
import json

from mmf.datasets.databases.annotation_database import AnnotationDatabase

## Inherit default MMF's AnnotationDatabase and update necessary
## function `load_annotation_db`
class CSIAnnotationDatabase(AnnotationDatabase):
    def load_annotation_db(self, path: str):
        db=df.read_csv(path)
        self.data = db
        self.start_idx =0


In [40]:

import copy
import json

import torch
from mmf.common.sample import Sample
from mmf.datasets.mmf_dataset import MMFDataset


class CSIFeaturesDataset(MMFDataset):
        
    def __init__(self, config, dataset, data_folder=None, *args, **kwargs):
        super().__init__("csi", config, dataset)
        self._data_folder = data_folder
        self._data_dir = os.path.join(get_mmf_root(), config.data_dir)

        if not self._data_folder:
            self._data_folder = os.path.join(self._data_dir, config.data_folder)

        if not os.path.exists(self._data_folder):
            raise RuntimeError(
                "Data folder {} for CSI is not present".format(self._data_folder)
            )

        # Check if the folder was actually extracted in the subfolder
        if config.data_folder in os.listdir(self._data_folder):
            self._data_folder = os.path.join(self._data_folder, config.data_folder)
            print(self._data_folder)

        if len(os.listdir(self._data_folder)) == 0:
            raise RuntimeError("CSI dataset folder is empty")

        self._load()
        
    def build_annotation_db(self):
        """
        We are overriding this function to return our custom AnnotationDatabase
        which we built specifically for OKVQA. Similarly, we can override
        build_image_db and build_feature_db for to implement custom versions of
        those for a particular dataset.
        """
        annotation_path = self._get_path_based_on_index(
            self.config, "annotations", self._index
        )
        print(annotation_path)
        return CSIAnnotationDatabase(self.config, annotation_path)

    def __getitem__(self, idx):
        sample_info = self.annotation_db[idx]
        current_sample = Sample()
        screenplay = sample_info["screenplay"]
        if isinstance(screenplay, list):
            screenplay = screenplay[0]
        processed_sentence = self.text_processor({"text": screenplay})

        current_sample.text = processed_sentence["text"]
        if "input_ids" in processed_sentence:
            current_sample.update(processed_sentence)

        if self._use_features is True:
            features = self.features_db[idx]
            if hasattr(self, "transformer_bbox_processor"):
                features["image_info_0"] = self.transformer_bbox_processor(
                    features["image_info_0"]
                )
            current_sample.update(features)

        processed = self.answer_processor({"answers": sample_info["label"]})
        current_sample.answers = processed["answers"]
        current_sample.targets = processed["answers_scores"]

        return current_sample


In [41]:
# Copyright (c) Facebook, Inc. and its affiliates.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#
print('here')
from mmf.common.registry import registry
from mmf.datasets.builders.csi.dataset import CSIFeaturesDataset
from mmf.datasets.mmf_dataset_builder import MMFDatasetBuilder


@registry.register_builder("csi")

class CSIDialogueBuilder(MMFDatasetBuilder):
    #ZOO_CONFIG_PATH = "/content/configs/zoo.yaml"
    def __init__(self, dataset_name="csi", dataset_class=CSIFeaturesDataset, *args, **kwargs):
        super().__init__(dataset_name, dataset_class,*args, **kwargs)


    @classmethod


    def config_path(cls):
        return "configs/datasets/csi/defaults.yaml"
    
    
    


here


In [46]:
from mmf.utils.build import build_dataset
from mmf.utils.configuration import Configuration

# Init configuration to register resolvers
#Configuration()
#dataset = build_dataset("csi", dataset_type="train")

_ = registry.unregister("csi")
